In [1]:
import os
os.environ['ANTHROPIC_API_KEY'] = "not making that fucking mistake"
# os.environ['ANTHROPIC_LOG'] = 'debug'

In [2]:
from claudette import *
sonnet35 = models[1]
opus3 = models[0]

In [3]:
def get_syntax_id(filename):
    extension = os.path.splitext(filename)[1].lower()
    syntax_map = {
        '.py': 'python',
        '.html': 'html',
        '.js': 'javascript',
        '.css': 'css',
        '.md': 'markdown',
        # add more mappings as needed
    }
    return syntax_map.get(extension, '')

def md_str(content: str, fname:str=None) -> str:
    syntax_id = get_syntax_id(fname)
    return f"### {fname}\n\n```{syntax_id}\n{content}\n```\n"

def f_md_str(filepath:str, custom_filename:str=None) -> str:
    """reads file as a markdown code snippet"""
    # print(f"f_md_str invoked with {filepath=}, {syntax_id=}, {custom_filename=}")
    with open(filepath, 'r') as f: content = f.read()
    filename = custom_filename or os.path.basename(filepath)
    return md_str(content, fname=filename)

def dir_md_str(directory:str) -> []:
    """reads directory as markdown code snippets. adds relative paths as headers for dir structure hints"""
    result = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if not os.path.basename(filename).startswith('.'):  # skip hidden files
                filepath = os.path.join(root, filename)
                rel_path = os.path.relpath(filepath, directory)
                result.append(f_md_str(filepath, rel_path))
    return result

def update_file(
    filepath:str, # The path of the file to update. if not provided
    content:str # The content to replace the contents with
) -> bool:
    """write content to file. tool for Claude to update files on my system"""
    with open(filepath, 'w') as o: o.write(content)
    return True

In [7]:
with open('prompts/system-prompt.md') as f: sp = f.read()
chat = Chat(opus3, sp=sp)
# use Chat(opus3 or sonnet35, sp=sp, tools=[update_file]) to inform Claude about the update_file tool

In [8]:
chat("hi Claude")

```xml
<thinking>
    <parse>
        simple greeting, no need for pleasantries
    </parse>
    <respond>
        acknowledge with equally terse reply
    </respond>
</thinking>
```

hello

<details>

- id: `msg_01GcNka6Bkdqf87Ab3B7q6v7`
- content: `[{'text': '```xml\n<thinking>\n    <parse>\n        simple greeting, no need for pleasantries\n    </parse>\n    <respond>\n        acknowledge with equally terse reply\n    </respond>\n</thinking>\n```\n\nhello', 'type': 'text'}]`
- model: `claude-3-opus-20240229`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'input_tokens': 469, 'output_tokens': 56, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0}`

</details>

In [9]:
attachment = f_md_str("prompts/system-prompt.md")
prompt = "what do you see?"
msg = mk_msg([attachment, prompt], cache=True)
chat(msg)

```xml
<thinking>
    <observe>
        I see a markdown file named system-prompt.md
        it contains instructions for how I should behave and format my responses
    </observe>
    <analyze>
        <metaprompt1>
            encourages brevity, technical detail, use of <thinking> tags, custom response format
        </metaprompt1>
        <metaprompt2>
            mentions I will be given tools to use wisely, but no specifics yet
        </metaprompt2>
    </analyze>
</thinking>
```

the file system-prompt.md contains:
- metaprompt1 with guidelines for my behavior and response format
- metaprompt2 hinting that I will be given tools to use
- an overall prompt structure with BEGIN/END tags

it seems to be a system prompt defining high-level parameters for our interaction. I will aim to follow the guidelines it lays out

<details>

- id: `msg_01XDWFvzQ4uTYH6y4bZ4p43c`
- content: `[{'text': '```xml\n<thinking>\n    <observe>\n        I see a markdown file named system-prompt.md\n        it contains instructions for how I should behave and format my responses\n    </observe>\n    <analyze>\n        <metaprompt1>\n            encourages brevity, technical detail, use of <thinking> tags, custom response format\n        </metaprompt1>\n        <metaprompt2>\n            mentions I will be given tools to use wisely, but no specifics yet\n        </metaprompt2>\n    </analyze>\n</thinking>\n```\n\nthe file system-prompt.md contains:\n- metaprompt1 with guidelines for my behavior and response format\n- metaprompt2 hinting that I will be given tools to use\n- an overall prompt structure with BEGIN/END tags\n\nit seems to be a system prompt defining high-level parameters for our interaction. I will aim to follow the guidelines it lays out', 'type': 'text'}]`
- model: `claude-3-opus-20240229`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'input_tokens': 1006, 'output_tokens': 220, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0}`

</details>